In [24]:
# Código do modelo matemático - Desafio Bootcamp - Enacom
# Autoria: Cassiano Tavares

In [25]:
# Verificação se todas as bibliotecas necessárias estão instaladas. Caso contrário, ~\
# esta célula irá realizar a instalação das bibliotecas ainda nao instaladas

import pip

def import_or_install(pac):
    try:
        __import__(pac)
    except ImportError:
        pip.main(['install', pac])    

bibliotecas = ["tkinter", "pandas", "numpy", "math", "datetime","io","pyomo","re","collections","plotly.express","plotly.graph_objects"]


for b in bibliotecas:
    import_or_install(b)

print("Bibliotecas instaladas")

Bibliotecas instaladas


In [26]:
# Carrega as bibliotecas do Python
from tkinter import E
import pandas as pd
import numpy as np
import math
#from google.colab import files
import datetime
import io
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import re
import sys                             
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go

In [27]:
# Define o arquivo com dados de entrada
nome_input = 'dados_desafio_2023.xlsx'

In [28]:
# Lê os dados de entrada como data frames

# Carteira de Investimentos (IGNORA LINHAS COM CÉLULAS EM BRANCO)
df_carteira_investimentos  = pd.read_excel(nome_input, sheet_name='dados_desafio').dropna()

print(df_carteira_investimentos)

   Opção                                     Descrição  \
0      1  Ampliação da capacidade do armazém ZPD em 5%   
1      2  Ampliação da capacidade do armazém MGL em 7%   
2      3                       Compra de empilhadeiras   
3      4                             Projetos de P&D I   
4      5                            Projetos de P&D II   
5      6               Aquisição de novos equipamentos   
6      7                   Capacitação de funcionários   
7      8    Ampliação da estrutura de carga rodoviária   

   Custo do investimento (R$)  Retorno esperado (R$)  
0                      470000                 410000  
1                      400000                 330000  
2                      170000                 140000  
3                      270000                 250000  
4                      340000                 320000  
5                      230000                 320000  
6                       50000                  90000  
7                      440000        

In [29]:
# Criação dos conjuntos


# Criação do conjunto de opções invenstimentos
dict_I = dict(enumerate(df_carteira_investimentos["Opção"]))
print("Este é o conjunto de opções investimentos")
print(dict_I)


Este é o conjunto de opções investimentos
{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8}


In [30]:
#Criação do parâmetro c_{i} - Custo de investimento do projeto


# Inicializa a matriz com zeros
c_i = [0 for e in dict_I.keys()]

# Preenche as posições de acordo com df_carteira_investimentos
for linha in df_carteira_investimentos.iterrows():
    
    # Extrai o conteúdo da linha atual
    dados_linha = list(linha[1])
    #print(dados_linha)


    # Define o índice do projeto de investimento
    try:
        # Procura o índice na lista do projeto de investimento
        i = list(dict_I.keys())[list(dict_I.values()).index(dados_linha[0])]  
        c_i[i]= dados_linha[2]        
    except:   
        # Projeto de investimento não encontrado
        i = -1


print(c_i)

[470000, 400000, 170000, 270000, 340000, 230000, 50000, 440000]


In [31]:
#Criação do parâmetro vpl_{i} - VPL do projeto


# Inicializa a matriz com zeros
vpl_i = [0 for e in dict_I.keys()]

# Preenche as posições de acordo com df_carteira_investimentos
for linha in df_carteira_investimentos.iterrows():
    
    # Extrai o conteúdo da linha atual
    dados_linha = list(linha[1])
    #print(dados_linha)


    # Define o índice do projeto de investimento
    try:
        # Procura o índice na lista do projeto de investimento
        i = list(dict_I.keys())[list(dict_I.values()).index(dados_linha[0])]  
        vpl_i[i]= dados_linha[3]        
    except:   
        # Projeto de investimento não encontrado
        i = -1


print(vpl_i)

[410000, 330000, 140000, 250000, 320000, 320000, 90000, 190000]


In [32]:
# Budget disponível para investimento
M = 1.E+6

In [33]:
# Criacao do modelo de otimização
model = pyo.ConcreteModel()


#Criação dos conjuntos do modelo de otimização

# Conjunto de opções invenstimentos
model.set_I = pyo.Set(initialize = dict_I.keys())
print("Este é o model.set_I")
print(dict_I.keys())

Este é o model.set_I
dict_keys([0, 1, 2, 3, 4, 5, 6, 7])


In [34]:
#Criação da variável de decisão


# Variaáel de decisão: 1 se o projeto de investimento é selecionado
model.Y = pyo.Var(model.set_I, within=Binary)
Y = model.Y 


In [35]:
# Função objetivo do modelo
model.exprobj = pyo.Expression()

#Função que calcula o VPL de todos os projetos
obj1 = sum(vpl_i[i]*Y[i] for i in model.set_I) 


# Função objetivo: 
model.exprobj =  obj1 

model.obj = pyo.Objective(expr= model.exprobj, sense = pyo.maximize)

In [36]:
#Restrição 02: Mochila

model.C2 = pyo.ConstraintList()
model.C2.add(expr=sum(Y[i]*c_i[i] for i in model.set_I) <= M)



In [37]:
#Restrição 03: Se o investimento 1 for selecionado, então o investimento 5 não deve ser

model.C3a = pyo.ConstraintList()
model.C3a.add(expr = Y[0] <= M*(1-Y[4]))

model.C3b= pyo.ConstraintList()
model.C3b.add(expr = Y[4] <= M*(Y[1]))

In [38]:
#Restrição 04: Se o investimento 2 for selecionado, então o investimento 4 também deve ser.

model.C4 = pyo.ConstraintList()
model.C4.add(expr = Y[1] <= Y[3])

In [39]:
# Escreve modelo em arquivo no formato LP
model.write('model.lp', io_options={'symbolic_solver_labels': True})

# Seleciona o solver

solver = pyo.SolverFactory('glpk')

solution = solver.solve(model, tee=True)

from pyomo.opt import SolverStatus, TerminationCondition

if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    print("Objective function value = ", model.obj())

elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
else:
    # something else is wrong
    print(str(solution.solver))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\CASSIA~1\AppData\Local\Temp\tmpdi60bi2d.glpk.raw --wglp
 C:\Users\CASSIA~1\AppData\Local\Temp\tmp2xiw_gfh.glpk.glp --cpxlp C:\Users\CASSIA~1\AppData\Local\Temp\tmpa9_jh7hn.pyomo.lp
Reading problem data from 'C:\Users\CASSIA~1\AppData\Local\Temp\tmpa9_jh7hn.pyomo.lp'...
C:\Users\CASSIA~1\AppData\Local\Temp\tmpa9_jh7hn.pyomo.lp:55: warning: lower bound of variable 'x1' redefined
C:\Users\CASSIA~1\AppData\Local\Temp\tmpa9_jh7hn.pyomo.lp:55: warning: upper bound of variable 'x1' redefined
5 rows, 9 columns, 15 non-zeros
8 integer variables, all of which are binary
63 lines were read
Writing problem data to 'C:\Users\CASSIA~1\AppData\Local\Temp\tmp2xiw_gfh.glpk.glp'...
46 lines were written
GLPK Integer Optimizer 5.0
5 rows, 9 columns, 15 non-zeros
8 integer variables, all of which are binary
Preprocessing...
2 constraint coefficient(s) were reduced
4 rows, 8 columns, 14 non-zeros
8 integer variable

In [40]:

#Impressão da variável Y - Variável de decisão: 1 se o projeto i é selecionado para investimento

for i in model.set_I:
    if pyo.value(Y[i]) > 1.E-3:
        print(f'Y[{i+1}] = {round(pyo.value(Y[i]),0)}')

Y[2] = 1.0
Y[4] = 1.0
Y[6] = 1.0
Y[7] = 1.0


In [41]:
# Lista construída iterativamente, que se tornará o DataFrame final de solução
df_sol_Y = []


for i in model.set_I:
    if pyo.value(Y[i]) > 1.E-3:

        # Valor da variável
        #valor = round(X[i][j][t],2)

        # Linha do DataFrame
        dict_linha = {

                    'Projeto': i ,
                        }

        # Guardando valor
        df_sol_Y.append(dict_linha)

# Criando DataFrame
df_sol_Y = pd.DataFrame(df_sol_Y)
df_sol_Y = df_sol_Y.sort_values(['Projeto'])

df_sol_Y.index = list(range(len(df_sol_Y)))


print(df_sol_Y)

   Projeto
0        1
1        3
2        5
3        6


In [42]:
#Criação do vetor com os nomes dos projetos


# Inicializa a matriz com zeros
nome_i = [0 for e in dict_I.keys()]

# Preenche as posições de acordo com df_carteira_investimentos
for linha in df_carteira_investimentos.iterrows():
    
    # Extrai o conteúdo da linha atual
    dados_linha = list(linha[1])
    #print(dados_linha)


    # Define o índice do projeto de investimento
    try:
        # Procura o índice na lista do projeto de investimento
        i = list(dict_I.keys())[list(dict_I.values()).index(dados_linha[0])]  
        nome_i[i]= dados_linha[1]        
    except:   
        # Projeto de investimento não encontrado
        i = -1


print(nome_i)

['Ampliação da capacidade do armazém ZPD em 5%', 'Ampliação da capacidade do armazém MGL em 7%', 'Compra de empilhadeiras', 'Projetos de P&D I', 'Projetos de P&D II', 'Aquisição de novos equipamentos', 'Capacitação de funcionários', 'Ampliação da estrutura de carga rodoviária']


In [43]:
#Manipulando DataFrame sol_Y para a construção de gráficos



#Adicionando as colunas de custo do projeto, vpl do projeto e descrição do projeto

# Coluna com os custos de cada projeto 
custo_projeto = []

# Coluna com o vpl de cada projeto
vpl_projeto = []

# Coluna com o nome de cada projeto
nome_projeto = []

# Preenche as posições de acordo com df_sol_X
for linha in df_sol_Y.iterrows():   
    # Extrai o conteúdo da linha atual
    dados_linha = list(linha[1])
    #print(dados_linha)
    custo_projeto.append(c_i[dados_linha[0]])
    vpl_projeto.append(vpl_i[dados_linha[0]])
    nome_projeto.append(nome_i[dados_linha[0]])

# Adicionanado a coluna com os custos de cada projeto no Dataframe da solução
df_sol_Y["Custo realizado"] = [elemento for elemento in custo_projeto]

# Adicionanado a coluna com o VPL de cada projeto no Dataframe da solução
df_sol_Y["VPL realizado"] = [elemento for elemento in vpl_projeto]

# Adicionanado a coluna com os nomes de cada projeto no Dataframe da solução
df_sol_Y["Nome do Projeto"] = [elemento for elemento in nome_projeto]

# Cálculo do total investido
total_investido = np.sum(custo_projeto)

print(total_investido)

# Calculo do VPL total esperado
vpl_total = np.sum(vpl_projeto)

print(vpl_total)

950000
990000


In [44]:
#Criando gráfico dos custos totais de investimento

fig_custo_projeto = px.bar(df_sol_Y, x="Nome do Projeto", y="Custo realizado", text = "Custo realizado", color = "Nome do Projeto",title="Custos dos investimentos realizados nos projetos")


# Total investido
fig_custo_projeto.add_trace(go.Scatter(x=df_sol_Y["Nome do Projeto"], y=[total_investido for nome in df_sol_Y["Nome do Projeto"] ],
                    line=dict(color='royalblue', width=2, dash='dot'),
                    name='Custo total investido'))
    
# Budget
fig_custo_projeto.add_trace(go.Scatter(x=df_sol_Y["Nome do Projeto"], y=[M for nome in df_sol_Y["Nome do Projeto"] ],
                    line = dict(color='firebrick', width=2, dash='dot'),
                    name='Budget para investimento'))



fig_custo_projeto.show()

In [45]:
#Criando gráfico dos VPLS totais de investimento




fig_vpl_total = px.pie(df_sol_Y, values='VPL realizado', 
                names='Nome do Projeto', title="VPL dos projetos")

fig_vpl_total.show()

fig_analise_vpl = px.bar(df_sol_Y, x="Nome do Projeto", y="VPL realizado", text = "VPL realizado", color = "Nome do Projeto",title="VPL dos projetos")



# Total investido
fig_analise_vpl.add_trace(go.Scatter(x=df_sol_Y["Nome do Projeto"], y=[total_investido for nome in df_sol_Y["Nome do Projeto"] ],
                    line=dict(color='royalblue', width=2, dash='dot'),
                    name='Custo total investido'))
    
# VPL retornado
fig_analise_vpl.add_trace(go.Scatter(x=df_sol_Y["Nome do Projeto"], y=[vpl_total for nome in df_sol_Y["Nome do Projeto"] ],
                    line = dict(color='firebrick', width=2, dash='dot'),
                    name='VPL retornado'))



fig_analise_vpl.show()